In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time

from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")  # sandbox 주요 요소 처리불가-> nosandbox 처리 가능

driver = webdriver.Chrome(options=options)

In [31]:
url='https://huggingface.co/papers'
driver.get(url)
time.sleep(0.2)

all_title=[]
all_news=[]
all_url=[]
all_author=[] 

titles=driver.find_elements(By.TAG_NAME,'h3')

for i in titles:
    all_title.append(i.text)
all_title=[i for i in all_title if i] 


for n in all_title:

    print(n)
    driver.find_element(By.PARTIAL_LINK_TEXT,n).click() #페이지 들어가기
    time.sleep(1)
    
    nourl=driver.current_url
    all_url.append(nourl)
    time.sleep(1)
    
    news=driver.find_element(By.TAG_NAME,'p')
    all_news.append(news.text)

  
    author=driver.find_element(By.CLASS_NAME,'whitespace-nowrap.underline')
    all_author.append(author.text)
        
    
    driver.back()


In [29]:
print(all_author)

['Jae Sung Park']


In [30]:
sumnews=[]
for t,n,u,a in zip(all_title,all_news,all_url,all_author):
    singles={'title':t,'news':n,'url':u,'author':a}
    sumnews.append(singles)

import pandas as pd
news_df=pd.DataFrame(sumnews)
news_df

,title,news,url,author
0,SparQ Attention: Bandwidth-Efficient LLM Infer...,Generative large language models (LLMs) have o...,https://huggingface.co/papers/2312.04985,Jae Sung Park


In [12]:
# import csv
# with open('test.txt','wt') as f:
#     wt=csv.writer(f)
#     wt.writerows(news_df)
    
news_df.to_excel('samtest.xlsx')

In [21]:
url='https://huggingface.co/papers/2312.04985'
driver.get(url)

sas=driver.find_elements(By.CLASS_NAME,'whitespace-nowrap.underline')

for i in sas:
    print(i.text)

Luka Ribar
Ivan Chelombiev
Luke Hudlass-Galley
Charlie Blake
Carlo Luschi
Douglas Orr
